In [1]:
import requests
import string

from utils.get_data import get_user, get_project, get_all_projects, get_all_users
from utils.metric_utils import intersection_metric, iou_metric
from utils.similarity_utils import cosine_similarity
from utils.text_utils import embedd_representation, get_user_roles_desc, get_user_text_desc, get_user_skill_desc, get_project_text_desc
from tqdm import tqdm
import pickle
import pandas as pd
import numpy as np
from scipy.spatial import distance
from typing import Dict, Any, List

In [2]:
get_project(1)

{'_id': '1',
 'type': 'hackathon',
 'projectName': 'Star Bear',
 'projectDescription': 'Проект для поиска единомышленников и обмена идеями',
 'isPublished': True,
 'projectTags': ['social', 'modern ui'],
 'projectOwnerId': '1',
 'existTeam': [{'userId': '1', 'category': 'Teamlead', 'skills': ['Excel']}],
 'lookingForTeam': [{'category': 'Frontend', 'skills': ['Angular']},
  {'category': 'Backend', 'skills': ['C#', 'Node js', 'Nest js']},
  {'category': 'Frontend', 'skills': ['React']}],
 'projectCostByAdmin': [{'rate': '', 'solution': '', 'publishDate': ''}],
 'isInnovated': False,
 'innovator': {'projectLink': '',
  'presentationFileId': '',
  'supportExperience': ['accelerator', 'grants']},
 'hackathon': {'targetTask': {'id': '1'}},
 'novatorOfMoscow': {'inn': '',
  'registration': '',
  'supportExperience': [''],
  'nomination': 'Проект будущего',
  'direction': 'Экология и охрана',
  'revenue': '',
  'projectReadinessStage': 'mvp',
  'projectLink': '',
  'presentationFileId': '',
 

In [123]:
get_all_projects()

[{'_id': '1',
  'type': 'hackathon',
  'projectName': 'Star Bear',
  'projectDescription': 'Проект для поиска единомышленников и обмена идеями',
  'isPublished': True,
  'projectTags': ['social', 'modern ui'],
  'projectOwnerId': '1',
  'existTeam': [{'userId': '1', 'category': 'Teamlead', 'skills': ['Excel']}],
  'lookingForTeam': [{'category': 'Frontend', 'skills': ['Angular']},
   {'category': 'Backend', 'skills': ['C#', 'Node js', 'Nest js']},
   {'category': 'Frontend', 'skills': ['React']}],
  'projectCostByAdmin': [{'rate': '', 'solution': '', 'publishDate': ''}],
  'isInnovated': False,
  'innovator': {'projectLink': '',
   'presentationFileId': '',
   'supportExperience': ['accelerator', 'grants']},
  'hackathon': {'targetTask': {'id': '1'}},
  'novatorOfMoscow': {'inn': '',
   'registration': '',
   'supportExperience': [''],
   'nomination': 'Проект будущего',
   'direction': 'Экология и охрана',
   'revenue': '',
   'projectReadinessStage': 'mvp',
   'projectLink': '',
   '

In [2]:
test_parquet = pd.read_parquet('/Users/andrey.krotkikh/Documents/embedds.parquet')

In [3]:
test_parquet.loc['В'].values

array([ 0.02952424,  0.03973465,  0.51920575,  0.2742361 , -0.56496364,
       -0.05899953,  0.15973805,  0.36940977, -0.28356385,  0.0317634 ,
        0.0545611 , -0.4064595 , -0.30602503,  0.1400957 , -0.42900386,
       -0.26042384, -0.35757917,  0.2937732 ,  0.6116437 , -0.30295795,
        0.02860113, -0.09477986,  0.12079441,  0.5180067 , -0.20671937,
       -0.1374913 , -0.19656545,  0.11080171,  0.00718931, -0.26903504,
        0.16081983, -0.15745698,  0.16667393, -0.38068062, -0.07165289,
        0.09804589,  0.06312373, -0.15158296,  0.19063157,  0.6958586 ,
        0.03368482,  0.130759  ,  0.17316923, -0.32077733, -0.0483497 ,
        0.23173176, -0.32908714, -0.07346074, -0.39619485,  0.19769314],
      dtype=float32)

In [4]:
test_parquet.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
",",0.165771,-0.517288,0.419655,-0.051957,-0.087263,-0.021959,0.089717,0.167934,-0.119870,-0.020433,...,-0.009044,-0.003489,0.187495,-0.144960,-0.020137,0.395344,-0.233079,0.025705,-0.408518,-0.224592
.,0.383446,0.011442,0.860179,0.129129,-0.266771,-0.042965,0.268558,0.321658,-0.243419,-0.057822,...,0.044906,0.143004,0.122937,-0.475062,0.100880,0.235484,-0.244221,-0.004340,-0.451731,0.027196
и,-0.010855,-0.695681,0.137943,0.078002,-0.112360,0.173395,0.444733,0.114993,0.025454,-0.001267,...,-0.142463,-0.062530,0.015674,-0.106922,-0.168451,0.215671,-0.326548,0.115889,-0.438935,-0.157844
в,0.072291,-0.373457,0.123495,0.219691,-0.415449,0.054333,-0.016865,0.366148,-0.209039,0.168200,...,-0.066749,0.003132,0.224285,-0.298141,-0.052664,0.423817,-0.368453,-0.214593,-0.278668,0.242669
</s>,0.804123,-0.131928,1.406710,0.330419,-0.397396,-0.131128,0.353531,0.324826,-0.467777,-0.049522,...,0.156702,0.000512,0.320597,-1.087240,-0.610243,0.274437,-0.429224,0.089688,-0.982359,0.325405


In [108]:
get_user(1)

{'_id': '1',
 'firstName': 'Иван',
 'secondName': 'Иванов',
 'lastName': 'Иванович',
 'birthDate': '26.10.1972',
 'country': 'Россия',
 'city': 'Москва',
 'citizenship': 'Российское',
 'gender': 'Мужской',
 'contacts': {'phone': '+95159195192',
  'email': 'ivanov@yandex.ru',
  'vk': '@ivanov',
  'telegram': '@ivanov',
  'skype': 'ivanov'},
 'education': {'startDate': '10.10.1990',
  'startEnd': '10.10.1996',
  'speciality': 'Компьютерные системы',
  'universityName': 'МГУ'},
 'jobInformation': {'employment': 'На себя',
  'jobExperience': '20',
  'achievements': ['Победа в хакатоне 2021',
   'Победа в хакатоне 2006',
   'Работал в СБЕР']},
 'skillInformation': {'categories': ['Капитан', 'Teamlead', 'Backend'],
  'skills': ['Excel', 'C#', '.net', 'Презентация', 'C++']},
 'intellectualProperty': {'isIntellectualPropertyAuthor': True,
  'numberOfDocument': '51295195129512'},
 'companyInformation': {'isCompanyOwner': True,
  'companyINN': '52151295912959'},
 'hackathon': {'countHackathons':

In [109]:
get_all_projects()

[{'_id': '1',
  'type': 'hackathon',
  'projectName': 'Star Bear',
  'projectDescription': 'Проект для поиска единомышленников и обмена идеями',
  'isPublished': True,
  'projectTags': ['social', 'modern ui'],
  'projectOwnerId': '1',
  'existTeam': [{'userId': '1', 'category': 'Teamlead', 'skills': ['Excel']}],
  'lookingForTeam': [{'category': 'Frontend', 'skills': ['Angular']},
   {'category': 'Backend', 'skills': ['C#', 'Node js', 'Nest js']},
   {'category': 'Frontend', 'skills': ['React']}],
  'projectCostByAdmin': [{'rate': '', 'solution': '', 'publishDate': ''}],
  'isInnovated': False,
  'innovator': {'projectLink': '',
   'presentationFileId': '',
   'supportExperience': ['accelerator', 'grants']},
  'hackathon': {'targetTask': {'id': '1'}},
  'novatorOfMoscow': {'inn': '',
   'registration': '',
   'supportExperience': [''],
   'nomination': 'Проект будущего',
   'direction': 'Экология и охрана',
   'revenue': '',
   'projectReadinessStage': 'mvp',
   'projectLink': '',
   '

In [110]:
def search_projects_for_specialist(user_id: int, input_df: pd.DataFrame) -> Dict[int, Dict[str, float]]:

    roles = get_user_roles_desc(user_id)
    skills = get_user_skill_desc(user_id)
    desc = get_user_text_desc(user_id)

    all_projects = get_all_projects()
    all_ids = [int(user['_id']) for user in all_projects]

    result_dict = {}

    for id in all_ids:
        id_metrics = {}
        looking_for_info = get_project(id)['lookingForTeam']
        for info in looking_for_info:
            role_metric = 1 if intersection_metric(roles, info['category']) > 0 else 0
            skills_metric = intersection_metric(skills, info['skills'])
            result_metric = role_metric * skills_metric
            id_metrics[info['category']] = result_metric
        project_desc = get_project_text_desc(id)
        id_metrics['semantic'] = cosine_similarity(embedd_representation(desc, input_df), embedd_representation(project_desc, input_df))
        result_dict[id] = id_metrics

    return result_dict

In [111]:
search_projects_for_specialist(1, test_parquet)

{1: {'Frontend': 0, 'Backend': 1, 'semantic': 0.6794743537902832},
 2: {'Frontend': 0, 'Backend': 2, 'semantic': 0.6167054176330566},
 3: {'Backend': 1, 'semantic': 0.5867804884910583}}

In [112]:
request_all_users = requests.get("https://space-bear-api.herokuapp.com/api/users/ml-get-all")
all_users_dict = request_all_users.json()
all_users = {int(user['_id']): user for user in all_users_dict}

request_all_projects = requests.get("https://space-bear-api.herokuapp.com/api/projects/ml-get-all")
all_projects_dict = request_all_projects.json()
all_projects = {int(project['_id']): project for project in all_projects_dict}

In [120]:
all_users[1]

{'_id': '1',
 'firstName': 'Иван',
 'secondName': 'Иванов',
 'lastName': 'Иванович',
 'birthDate': '26.10.1972',
 'country': 'Россия',
 'city': 'Москва',
 'citizenship': 'Российское',
 'gender': 'Мужской',
 'contacts': {'phone': '+95159195192',
  'email': 'ivanov@yandex.ru',
  'vk': '@ivanov',
  'telegram': '@ivanov',
  'skype': 'ivanov'},
 'education': {'startDate': '10.10.1990',
  'startEnd': '10.10.1996',
  'speciality': 'Компьютерные системы',
  'universityName': 'МГУ'},
 'jobInformation': {'employment': 'На себя',
  'jobExperience': '20',
  'achievements': ['Победа в хакатоне 2021',
   'Победа в хакатоне 2006',
   'Работал в СБЕР']},
 'skillInformation': {'categories': ['Капитан', 'Teamlead', 'Backend'],
  'skills': ['Excel', 'C#', '.net', 'Презентация', 'C++']},
 'intellectualProperty': {'isIntellectualPropertyAuthor': True,
  'numberOfDocument': '51295195129512'},
 'companyInformation': {'isCompanyOwner': True,
  'companyINN': '52151295912959'},
 'hackathon': {'countHackathons':

In [116]:
list(all_users.values())

[{'_id': '1',
  'firstName': 'Иван',
  'secondName': 'Иванов',
  'lastName': 'Иванович',
  'birthDate': '26.10.1972',
  'country': 'Россия',
  'city': 'Москва',
  'citizenship': 'Российское',
  'gender': 'Мужской',
  'contacts': {'phone': '+95159195192',
   'email': 'ivanov@yandex.ru',
   'vk': '@ivanov',
   'telegram': '@ivanov',
   'skype': 'ivanov'},
  'education': {'startDate': '10.10.1990',
   'startEnd': '10.10.1996',
   'speciality': 'Компьютерные системы',
   'universityName': 'МГУ'},
  'jobInformation': {'employment': 'На себя',
   'jobExperience': '20',
   'achievements': ['Победа в хакатоне 2021',
    'Победа в хакатоне 2006',
    'Работал в СБЕР']},
  'skillInformation': {'categories': ['Капитан', 'Teamlead', 'Backend'],
   'skills': ['Excel', 'C#', '.net', 'Презентация', 'C++']},
  'intellectualProperty': {'isIntellectualPropertyAuthor': True,
   'numberOfDocument': '51295195129512'},
  'companyInformation': {'isCompanyOwner': True,
   'companyINN': '52151295912959'},
  'h

In [119]:
request = requests.get("https://space-bear-api.herokuapp.com/api/projects/ml-get-all")
json_dict = request.json()
json_dict[0]

{'_id': '1',
 'type': 'hackathon',
 'projectName': 'Star Bear',
 'projectDescription': 'Проект для поиска единомышленников и обмена идеями',
 'isPublished': True,
 'projectTags': ['social', 'modern ui'],
 'projectOwnerId': '1',
 'existTeam': [{'userId': '1', 'category': 'Teamlead', 'skills': ['Excel']}],
 'lookingForTeam': [{'category': 'Frontend', 'skills': ['Angular']},
  {'category': 'Backend', 'skills': ['C#', 'Node js', 'Nest js']},
  {'category': 'Frontend', 'skills': ['React']}],
 'projectCostByAdmin': [{'rate': '', 'solution': '', 'publishDate': ''}],
 'isInnovated': False,
 'innovator': {'projectLink': '',
  'presentationFileId': '',
  'supportExperience': ['accelerator', 'grants']},
 'hackathon': {'targetTask': {'id': '1'}},
 'novatorOfMoscow': {'inn': '',
  'registration': '',
  'supportExperience': [''],
  'nomination': 'Проект будущего',
  'direction': 'Экология и охрана',
  'revenue': '',
  'projectReadinessStage': 'mvp',
  'projectLink': '',
  'presentationFileId': '',
 